# Traitement des états de sections de Marolles-En-Brie

## Pré-requis et documentation

Documentation:
- [Doc de l'API Arkindex](https://doc.arkindex.org/overview/api/)
- [Doc du client Python de bas-niveau](https://pypi.org/project/arkindex-client/)
- [Doc de Arkindex CLI (haut niveau)](https://cli.arkindex.org/)

A installer :

In [ ]:
!pip install arkindex-cli

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.7/239.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 43.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Login

In [ ]:
!arkindex login --host arkindex.teklia.com --email solenn.tual@ign.fr

Loading API client… Done!
Password: 
Authentication successful
Slug to save profile as (default): prod
Set this profile as the default? (yes): yes


## Statistiques du dataset

In [ ]:
import pandas as pd
import os
import plotly.express as px
import json
from arkindex import ArkindexClient

In [ ]:
cli = ArkindexClient("59285827-bba3-42d4-99c7-e2436fe19f94", base_url='https://arkindex.teklia.com')
cli = ArkindexClient('cdf9b285c9da6e82cddbf18aa070dc471a13facb')

In [ ]:
#Retrieve corpus (project)
corpus = cli.request('ListCorpus')
for c in corpus:
    #print(c)
    if c["name"] == 'EPITA-IGN | Registre Napoléonien':
      corpus_id = c["id"]
      print(c)
      print(json.dumps(c, indent=2))

{'id': '59285827-bba3-42d4-99c7-e2436fe19f94', 'name': 'EPITA-IGN | Registre Napoléonien', 'description': 'Projet de Solenn', 'public': False, 'rights': ['read', 'write', 'admin'], 'types': [{'id': 'a2718e04-c140-47c2-886d-42cf5757bf04', 'slug': 'page', 'display_name': 'Page', 'folder': False, 'color': 'b828b3'}, {'id': 'fede5d95-e562-4f33-97e2-1fe18d20e2f1', 'slug': 'text_line', 'display_name': 'Text line', 'folder': False, 'color': 'f4f740'}, {'id': '8e98a93d-992c-4a32-a6da-2bd17abcb3a4', 'slug': 'text_zone', 'display_name': 'Text zone', 'folder': False, 'color': '0ac4be'}, {'id': '151d522f-09cf-4a6e-9874-d39f31b4512f', 'slug': 'double_page', 'display_name': 'Double page', 'folder': False, 'color': 'b8284c'}, {'id': '064c8649-08ce-4c59-b60b-ffa8a7802376', 'slug': 'folder', 'display_name': 'Folder', 'folder': True, 'color': 'f5fff5'}, {'id': '3da0de23-ea2d-48b7-9f6f-4b9120b7c911', 'slug': 'tab_header', 'display_name': 'Table Header', 'folder': False, 'color': '233990'}, {'id': 'e36317

In [ ]:
import pandas as pd
import re

def cleantable(elements):
  res = []

  for e in elements["results"]:
    rs1 = {}
    rs1["lineid"] = e["id"]
    rs1["linename"] = int(e["name"])
    rs1["coords"] = e["zone"]["polygon"]
    rs1["image_id"] = e["zone"]["image"]["id"]
    rs1["image_name"] = e["zone"]["image"]["path"]
    rs1["page_cote"] = e["zone"]["image"]["path"].replace('CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFRAD094_3P_000387_01%2F','')
    rs1["page_cote"] = rs1["page_cote"].replace('.jpg','')
    rs1["page_cote_num"] = rs1["page_cote"].replace('FRAD094_3P_000387_01_','')
    rs1["page_cote_num"] = int(re.sub('^0+','',rs1["page_cote_num"]))
    rs1["image_url"] = e["zone"]["url"]
    rs1["image_width"] = e["zone"]["image"]["width"]
    rs1["image_height"] = e["zone"]["image"]["height"]

    if rs1["page_cote_num"] > 2 and rs1["page_cote_num"] < 20:
        rs1["section_id"] = 'A'
        rs1["section_name"] = "dite des Bruyères"
    elif rs1["page_cote_num"] > 20 and rs1["page_cote_num"] < 41:
        rs1["section_id"] = 'B'
        rs1["section_name"] = "dite de la butte au berger"
    elif rs1["page_cote_num"] > 41 and rs1["page_cote_num"] < 76:
        rs1["section_id"] = 'C'
        rs1["section_name"] = "dite du Village"
    elif rs1["page_cote_num"] > 76:
        rs1["section_id"] = 'D'
        rs1["section_name"] = "dite du pont de brie"
    res.append(rs1)
  return res

In [ ]:
#Retrieve table_line from Marolles additional dataset
elements = cli.request('ListElementChildren',id="fdbd71ea-4217-41eb-9c80-58a928b18c63",type="tab_line_marolles",recursive=True,folder=False,page_size=500,page=1)
resaddmarolles = cleantable(elements)
dfaddmarolles = pd.DataFrame.from_dict(resaddmarolles)

elements2 = cli.request('ListElementChildren',id="fdbd71ea-4217-41eb-9c80-58a928b18c63",type="tab_line",recursive=True,folder=False,page_size=500,page=1)
resmaindataset = cleantable(elements2)
dfmaindataset = pd.DataFrame.from_dict(resmaindataset)

fulltab = pd.concat([dfaddmarolles, dfmaindataset])
results = fulltab.sort_values(by=['page_cote','linename'])
results = results.reset_index(drop=True)
display(results)

,lineid,linename,coords,image_id,image_name,page_cote,page_cote_num,image_url,image_width,image_height,section_id,section_name
0,e92ddd2b-583f-4234-8a52-9f8745cda6c2,1,"[[223, 792], [223, 1015], [2520, 1015], [2520,...",c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0003,3,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,4013,A,dite des Bruyères
1,99407b12-f463-4b99-a420-2962667d0fef,2,"[[219, 973], [219, 1129], [2525, 1129], [2525,...",c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0003,3,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,4013,A,dite des Bruyères
2,cd655441-bac8-4d0b-9c8c-56233814fbb2,3,"[[219, 1110], [219, 1285], [2518, 1285], [2518...",c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0003,3,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,4013,A,dite des Bruyères
3,9695ae1d-09e7-4493-8a10-0c536d4b53d6,4,"[[215, 1272], [215, 1393], [2518, 1393], [2518...",c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0003,3,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,4013,A,dite des Bruyères
4,c69e476c-065d-4833-8730-11fa475e280e,5,"[[222, 1387], [222, 1562], [2515, 1562], [2515...",c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0003,3,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,4013,A,dite des Bruyères
...,...,...,...,...,...,...,...,...,...,...,...,...
615,5f4cdf48-9c20-4d7f-ac13-bd3fbe08503d,5,"[[326, 1369], [326, 1570], [2636, 1570], [2636...",302729ca-34b8-429d-8d32-cf8da9995c98,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0100,100,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2787,4082,D,dite du pont de brie
616,4a186c8f-51db-482b-a129-7f9a1d87364e,6,"[[329, 1527], [329, 1683], [2639, 1683], [2639...",302729ca-34b8-429d-8d32-cf8da9995c98,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0100,100,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2787,4082,D,dite du pont de brie
617,fce30d28-1fd2-4d57-9bdd-f4b990a936fe,7,"[[326, 1651], [326, 1796], [2633, 1796], [2633...",302729ca-34b8-429d-8d32-cf8da9995c98,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0100,100,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2787,4082,D,dite du pont de brie
618,33680098-0604-4637-a891-675c9cee8878,8,"[[320, 1774], [320, 1913], [2633, 1913], [2633...",302729ca-34b8-429d-8d32-cf8da9995c98,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0100,100,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2787,4082,D,dite du pont de brie


In [ ]:
# @title Supprimer les transcriptions
#import time

#Delete transcription
#for i in range(1):
  #time.sleep(3)
  #tt = cli.request('ListTranscriptions',id="0846dcd6-2a9f-40cb-89e5-ff261c59365d",recursive=True,element_type="tab_line")
  #transcript = tt["results"]
  #print(len(transcript))
  #for e in transcript:
    #id = e["id"]
    #print(id)
    #cli.request('DestroyTranscription',id=id)

In [ ]:
annotatedlines = cli.request('ListTranscriptions',id="0846dcd6-2a9f-40cb-89e5-ff261c59365d",recursive=True,element_type="tab_line")
annotatedlines["count"]

2507

In [ ]:
# PAS ADAPTE POUR l'API !!
#import json

# Use pd.json_normalize to convert the JSON to a DataFrame
#elementsT = pd.json_normalize(annotatedlines['results'],
                     meta=['id', 'text', 'orientation', 'confidence', 'orientation', 'worker_version_id',['worker_run','id'],['worker_run','summary'],['element', 'id'],['element','type'],['element','name'],['element','zone','id'],
                           ['element','zone','polygon'],['element','rotation_angle'],['element','mirrored']])
#elementsT = elementsT[['element.id','id','text','confidence']]
# Rename the columns for clarity
#elementsT.columns = ['lineid','transcriptid', 'text', 'text_confidence']
# Display the DataFrame
#display(elementsT)

IndentationError: unexpected indent (<ipython-input-10-7ca205e3b6ab>, line 6)

In [ ]:
import pandas as pd
import re

def cleantranscription(elementslinesdf):
  res = []

  for i in range(len(elementslinesdf)):
    lineid = elementslinesdf['lineid'].iloc[i]
    lineTRrequest = cli.request('ListTranscriptions',id=lineid)
    for e in lineTRrequest["results"]:
      rs1 = {}
      rs1["lineid"] = lineid
      rs1["transcriptid"] = e["id"]
      rs1["text"] = e["text"]
      rs1["text_confidence"] = e["confidence"]
      res.append(rs1)
  return res

In [ ]:
elementsT = cleantranscription(results)
dfelementsT = pd.DataFrame.from_dict(elementsT)
display(dfelementsT)

,lineid,transcriptid,text,text_confidence
0,e92ddd2b-583f-4234-8a52-9f8745cda6c2,55f95cd0-786d-44b2-8ddb-c59846c72b2e,Bois au pont→d'Encre 1 Le Prince de Neuchâtel→...,1.0
1,99407b12-f463-4b99-a420-2962667d0fef,a5e9469a-1cc3-4d44-bec9-062573e034f1,§ 2 Idem Id,1.0
2,cd655441-bac8-4d0b-9c8c-56233814fbb2,0cba8885-adb6-43c3-896c-c3307413a37c,Bois de la→Grippe 3 Domaine de la→Couronne Pâture,1.0
3,9695ae1d-09e7-4493-8a10-0c536d4b53d6,df0d2251-755f-48d2-a15d-ecc239d3d684,Les uzelles de→Marolles 4 Idem Id,1.0
4,c69e476c-065d-4833-8730-11fa475e280e,9b69ba2d-0d03-441d-8b93-fabe8e92ddbf,Bois de la→grippe 5 Idem Bois,1.0
...,...,...,...,...
532,5f4cdf48-9c20-4d7f-ac13-bd3fbe08503d,00efbad7-ab5a-40ec-8dae-af49ab68f33c,§ 225 Coudray auguste→andré id,1.0
533,4a186c8f-51db-482b-a129-7f9a1d87364e,adbd7fbb-53b5-4a9e-b088-1dbe21d82ea4,§ 226 huguet Louis id,1.0
534,fce30d28-1fd2-4d57-9bdd-f4b990a936fe,5415ac66-4c66-42cc-8f84-8d67e2f193b0,§ 227 Berthier id,1.0
535,33680098-0604-4637-a891-675c9cee8878,8f972b45-ff29-430b-92da-8dfc853dab18,§ 228 Tartet Veuve Terre,1.0


In [ ]:
def getstructuredtext(elementsTRdf):
  res = []

  for i in range(len(elementsTRdf)):
    TRid = elementsTRdf['transcriptid'].iloc[i]
    ENTrequest = cli.request('ListTranscriptionEntities',id=TRid)
    for e in ENTrequest["results"]:
      rs1 = {}
      rs1["transcriptid"] = TRid
      rs1["entities_count"] = ENTrequest["count"]
      rs1["entity_id"] = e['entity']["id"]
      rs1["entity_text"] = e['entity']["name"]
      rs1["entity_type"] = e['entity']["type"]["name"]
      res.append(rs1)
  return res

In [ ]:
elementsENT = getstructuredtext(dfelementsT)
dfelementsENT = pd.DataFrame.from_dict(elementsENT)
display(dfelementsENT)

,transcriptid,entities_count,entity_id,entity_text,entity_type
0,55f95cd0-786d-44b2-8ddb-c59846c72b2e,4,5230b28e-0fd9-403a-8cd9-ba8d3428a91f,Bois au pont→d'Encre,lieu-dit
1,55f95cd0-786d-44b2-8ddb-c59846c72b2e,4,1268c53b-962f-450e-8713-6cf7d4aa2f88,1,numero_parcelle
2,55f95cd0-786d-44b2-8ddb-c59846c72b2e,4,c19adf3a-7888-4363-804d-c3161806f0df,"Le Prince de Neuchâtel→×Vauguyon→Bourgeois, ma...",identite
3,55f95cd0-786d-44b2-8ddb-c59846c72b2e,4,e1b7d3aa-89a1-4218-91ff-b75e678e7de5,Bois,nature
4,a5e9469a-1cc3-4d44-bec9-062573e034f1,4,09c877bf-10c0-4913-b316-0b8977ed3ce3,§,lieu-dit
...,...,...,...,...,...
2144,8f972b45-ff29-430b-92da-8dfc853dab18,4,68fed831-df5b-47bc-9fbf-5211b9fd5c09,Terre,nature
2145,cba8c592-b6f1-4384-968f-7bf03ee2e16e,4,281d04ab-050c-4518-b853-c5a5b7716639,§,lieu-dit
2146,cba8c592-b6f1-4384-968f-7bf03ee2e16e,4,f004a201-1911-4a4e-952d-ade820103be4,229,numero_parcelle
2147,cba8c592-b6f1-4384-968f-7bf03ee2e16e,4,3b5a792b-a1be-4df6-bfcd-e190a11d0b73,C↑ie↓ des ch↑ins↓ de fer de l'Est,identite


In [ ]:
TR_ENT = pd.merge(dfelementsENT, dfelementsT, on=["transcriptid"],how="inner")
display(TR_ENT)

,transcriptid,entities_count,entity_id,entity_text,entity_type,lineid,text,text_confidence
0,55f95cd0-786d-44b2-8ddb-c59846c72b2e,4,5230b28e-0fd9-403a-8cd9-ba8d3428a91f,Bois au pont→d'Encre,lieu-dit,e92ddd2b-583f-4234-8a52-9f8745cda6c2,Bois au pont→d'Encre 1 Le Prince de Neuchâtel→...,1.0
1,55f95cd0-786d-44b2-8ddb-c59846c72b2e,4,1268c53b-962f-450e-8713-6cf7d4aa2f88,1,numero_parcelle,e92ddd2b-583f-4234-8a52-9f8745cda6c2,Bois au pont→d'Encre 1 Le Prince de Neuchâtel→...,1.0
2,55f95cd0-786d-44b2-8ddb-c59846c72b2e,4,c19adf3a-7888-4363-804d-c3161806f0df,"Le Prince de Neuchâtel→×Vauguyon→Bourgeois, ma...",identite,e92ddd2b-583f-4234-8a52-9f8745cda6c2,Bois au pont→d'Encre 1 Le Prince de Neuchâtel→...,1.0
3,55f95cd0-786d-44b2-8ddb-c59846c72b2e,4,e1b7d3aa-89a1-4218-91ff-b75e678e7de5,Bois,nature,e92ddd2b-583f-4234-8a52-9f8745cda6c2,Bois au pont→d'Encre 1 Le Prince de Neuchâtel→...,1.0
4,a5e9469a-1cc3-4d44-bec9-062573e034f1,4,09c877bf-10c0-4913-b316-0b8977ed3ce3,§,lieu-dit,99407b12-f463-4b99-a420-2962667d0fef,§ 2 Idem Id,1.0
...,...,...,...,...,...,...,...,...
2144,8f972b45-ff29-430b-92da-8dfc853dab18,4,68fed831-df5b-47bc-9fbf-5211b9fd5c09,Terre,nature,33680098-0604-4637-a891-675c9cee8878,§ 228 Tartet Veuve Terre,1.0
2145,cba8c592-b6f1-4384-968f-7bf03ee2e16e,4,281d04ab-050c-4518-b853-c5a5b7716639,§,lieu-dit,043f8ba3-a2ff-43c4-ab45-107e225d3ebf,§ 229 C↑ie↓ des ch↑ins↓ de fer de l'Est terre,1.0
2146,cba8c592-b6f1-4384-968f-7bf03ee2e16e,4,f004a201-1911-4a4e-952d-ade820103be4,229,numero_parcelle,043f8ba3-a2ff-43c4-ab45-107e225d3ebf,§ 229 C↑ie↓ des ch↑ins↓ de fer de l'Est terre,1.0
2147,cba8c592-b6f1-4384-968f-7bf03ee2e16e,4,3b5a792b-a1be-4df6-bfcd-e190a11d0b73,C↑ie↓ des ch↑ins↓ de fer de l'Est,identite,043f8ba3-a2ff-43c4-ab45-107e225d3ebf,§ 229 C↑ie↓ des ch↑ins↓ de fer de l'Est terre,1.0


In [ ]:
PL_TR_ENT = pd.merge(TR_ENT, results, on=["lineid"],how="inner")
display(PL_TR_ENT)

,transcriptid,entities_count,entity_id,entity_text,entity_type,lineid,text,text_confidence,linename,coords,image_id,image_name,page_cote,page_cote_num,image_url,image_width,image_height,section_id,section_name
0,55f95cd0-786d-44b2-8ddb-c59846c72b2e,4,5230b28e-0fd9-403a-8cd9-ba8d3428a91f,Bois au pont→d'Encre,lieu-dit,e92ddd2b-583f-4234-8a52-9f8745cda6c2,Bois au pont→d'Encre 1 Le Prince de Neuchâtel→...,1.0,1,"[[223, 792], [223, 1015], [2520, 1015], [2520,...",c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0003,3,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,4013,A,dite des Bruyères
1,55f95cd0-786d-44b2-8ddb-c59846c72b2e,4,1268c53b-962f-450e-8713-6cf7d4aa2f88,1,numero_parcelle,e92ddd2b-583f-4234-8a52-9f8745cda6c2,Bois au pont→d'Encre 1 Le Prince de Neuchâtel→...,1.0,1,"[[223, 792], [223, 1015], [2520, 1015], [2520,...",c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0003,3,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,4013,A,dite des Bruyères
2,55f95cd0-786d-44b2-8ddb-c59846c72b2e,4,c19adf3a-7888-4363-804d-c3161806f0df,"Le Prince de Neuchâtel→×Vauguyon→Bourgeois, ma...",identite,e92ddd2b-583f-4234-8a52-9f8745cda6c2,Bois au pont→d'Encre 1 Le Prince de Neuchâtel→...,1.0,1,"[[223, 792], [223, 1015], [2520, 1015], [2520,...",c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0003,3,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,4013,A,dite des Bruyères
3,55f95cd0-786d-44b2-8ddb-c59846c72b2e,4,e1b7d3aa-89a1-4218-91ff-b75e678e7de5,Bois,nature,e92ddd2b-583f-4234-8a52-9f8745cda6c2,Bois au pont→d'Encre 1 Le Prince de Neuchâtel→...,1.0,1,"[[223, 792], [223, 1015], [2520, 1015], [2520,...",c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0003,3,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,4013,A,dite des Bruyères
4,a5e9469a-1cc3-4d44-bec9-062573e034f1,4,09c877bf-10c0-4913-b316-0b8977ed3ce3,§,lieu-dit,99407b12-f463-4b99-a420-2962667d0fef,§ 2 Idem Id,1.0,2,"[[219, 973], [219, 1129], [2525, 1129], [2525,...",c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0003,3,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,4013,A,dite des Bruyères
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2144,8f972b45-ff29-430b-92da-8dfc853dab18,4,68fed831-df5b-47bc-9fbf-5211b9fd5c09,Terre,nature,33680098-0604-4637-a891-675c9cee8878,§ 228 Tartet Veuve Terre,1.0,8,"[[320, 1774], [320, 1913], [2633, 1913], [2633...",302729ca-34b8-429d-8d32-cf8da9995c98,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0100,100,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2787,4082,D,dite du pont de brie
2145,cba8c592-b6f1-4384-968f-7bf03ee2e16e,4,281d04ab-050c-4518-b853-c5a5b7716639,§,lieu-dit,043f8ba3-a2ff-43c4-ab45-107e225d3ebf,§ 229 C↑ie↓ des ch↑ins↓ de fer de l'Est terre,1.0,9,"[[316, 1903], [316, 2101], [2630, 2101], [2630...",302729ca-34b8-429d-8d32-cf8da9995c98,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0100,100,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2787,4082,D,dite du pont de brie
2146,cba8c592-b6f1-4384-968f-7bf03ee2e16e,4,f004a201-1911-4a4e-952d-ade820103be4,229,numero_parcelle,043f8ba3-a2ff-43c4-ab45-107e225d3ebf,§ 229 C↑ie↓ des ch↑ins↓ de fer de l'Est terre,1.0,9,"[[316, 1903], [316, 2101], [2630, 2101], [2630...",302729ca-34b8-429d-8d32-cf8da9995c98,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,FRAD094_3P_000387_01_0100,100,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2787,4082,D,dite du pont de brie
2147,cba8c592-b6f1-4384-968f-7bf03ee2e16e,4,3b5a792b-a1be-4df6-bfcd-e190a11d0b73,C↑ie↓ des ch↑ins↓ de fer de l'Est,identite,043f8ba3-a2ff-43c4-ab45-107e225d3ebf,§ 229 C↑ie↓ des ch↑ins↓ de fer de l'Est terre,1.0,9,"[[316, 1903], [316, 210

In [ ]:
columns = PL_TR_ENT.columns
print(columns)

Index(['transcriptid', 'entities_count', 'entity_id', 'entity_text',
       'entity_type', 'lineid', 'text', 'text_confidence', 'linename',
       'coords', 'image_id', 'image_name', 'page_cote', 'page_cote_num',
       'image_url', 'image_width', 'image_height', 'section_id',
       'section_name'],
      dtype='object')


In [ ]:
reorder_col = ['entity_id','entity_text','entity_type','section_id', 'section_name','page_cote','page_cote_num','lineid','linename','coords','transcriptid','text','text_confidence','entities_count','image_id',
               'image_name','image_url','image_width','image_height']
cleantab = PL_TR_ENT[reorder_col]
cleantab = cleantab.sort_values(by=['page_cote_num','linename'])
#cleantab = cleantab.reset_index(drop=True)
cleantab.head(10)
cleantab.to_csv(encoding='utf8')

',entity_id,entity_text,entity_type,section_id,section_name,page_cote,page_cote_num,lineid,linename,coords,transcriptid,text,text_confidence,entities_count,image_id,image_name,image_url,image_width,image_height\n0,5230b28e-0fd9-403a-8cd9-ba8d3428a91f,Bois au pont→d\'Encre,lieu-dit,A,dite des Bruyères,FRAD094_3P_000387_01_0003,3,e92ddd2b-583f-4234-8a52-9f8745cda6c2,1,"[[223, 792], [223, 1015], [2520, 1015], [2520, 792], [223, 792]]",55f95cd0-786d-44b2-8ddb-c59846c72b2e,"Bois au pont→d\'Encre 1 Le Prince de Neuchâtel→×Vauguyon→Bourgeois, maire± Bois",1.0,4,c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFRAD094_3P_000387_01%2FFRAD094_3P_000387_01_0003.jpg,"https://iiif.geohistoricaldata.org/iiif/2/CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFRAD094_3P_000387_01%2FFRAD094_3P_000387_01_0003.jpg/223,792,2297,223/full/0/default.jpg",2665,4013\n1,1268c53b-962f-450e-8713-6cf7d4aa2f88,1,numero_parcelle,A,dite des Bruyères,FRAD094_3P_000387_01_0003,3,e92ddd2b-583

## Création d'une table agrégée avec une ligne par parcelle

In [ ]:
num_p = cleantab[cleantab['entity_type'] == 'numero_parcelle']
nump_select = num_p[['lineid','entity_id','entity_text']]
nump_select = nump_select.rename(columns={"entity_id": "numero_parcelle_srcid", "entity_text": "numero_parcelle"})
display(nump_select)

,lineid,numero_parcelle_srcid,numero_parcelle
1,e92ddd2b-583f-4234-8a52-9f8745cda6c2,1268c53b-962f-450e-8713-6cf7d4aa2f88,1
5,99407b12-f463-4b99-a420-2962667d0fef,3f59a7ee-5556-4938-809f-ec6fde6c1c62,2
9,cd655441-bac8-4d0b-9c8c-56233814fbb2,1fcf647c-61cd-4542-b38b-541ede5be2af,3
13,9695ae1d-09e7-4493-8a10-0c536d4b53d6,845c9397-711e-4973-afdb-7010014da54a,4
17,c69e476c-065d-4833-8730-11fa475e280e,1f268a1d-fc4a-483c-82bd-6a30590ded95,5
...,...,...,...
2130,5f4cdf48-9c20-4d7f-ac13-bd3fbe08503d,0a2dc928-caba-4fb7-a9e3-6f2daa8c7e0e,225
2134,4a186c8f-51db-482b-a129-7f9a1d87364e,aac242a1-9f27-4562-af98-4c396d2fc95d,226
2138,fce30d28-1fd2-4d57-9bdd-f4b990a936fe,446ba364-32a7-443f-a1b7-ecedf0898a8d,227
2142,33680098-0604-4637-a891-675c9cee8878,3bab9126-43a5-4538-84de-3717d7787690,228


In [ ]:
identite_p = cleantab[cleantab['entity_type'] == 'identite']
identitep_select = identite_p[['lineid','entity_id','entity_text']]
identitep_select = identitep_select.rename(columns={"entity_id": "proprietaire_srcid", "entity_text": "proprietaire"})
display(identitep_select)

,lineid,proprietaire_srcid,proprietaire
2,e92ddd2b-583f-4234-8a52-9f8745cda6c2,c19adf3a-7888-4363-804d-c3161806f0df,"Le Prince de Neuchâtel→×Vauguyon→Bourgeois, ma..."
6,99407b12-f463-4b99-a420-2962667d0fef,27acbd42-bfce-46d9-afec-856de6a9cbcd,Idem
10,cd655441-bac8-4d0b-9c8c-56233814fbb2,7577a18e-e2d9-415b-b28f-eed31a3dbe9f,Domaine de la→Couronne
14,9695ae1d-09e7-4493-8a10-0c536d4b53d6,d046417b-0a6d-4ada-8653-88d211425f53,Idem
18,c69e476c-065d-4833-8730-11fa475e280e,0d468201-166f-49ef-ae12-f970b9c7de94,Idem
...,...,...,...
2131,5f4cdf48-9c20-4d7f-ac13-bd3fbe08503d,955b6f67-d7bb-4819-807b-c0d357c1830d,Coudray auguste→andré
2135,4a186c8f-51db-482b-a129-7f9a1d87364e,18f05b43-fd40-49d8-ad7c-95f947ef45d0,huguet Louis
2139,fce30d28-1fd2-4d57-9bdd-f4b990a936fe,c8a78be8-59e2-4461-ab5a-35b5ba3147f1,Berthier
2143,33680098-0604-4637-a891-675c9cee8878,38cd34fa-b3fc-4a42-82f5-e1157b3afc4b,Tartet Veuve


In [ ]:
nature_p = cleantab[cleantab['entity_type'] == 'nature']
naturep_select = nature_p[['lineid','entity_id','entity_text']]
naturep_select = naturep_select.rename(columns={"entity_id": "nature_srcid", "entity_text": "nature"})
display(naturep_select)

,lineid,nature_srcid,nature
3,e92ddd2b-583f-4234-8a52-9f8745cda6c2,e1b7d3aa-89a1-4218-91ff-b75e678e7de5,Bois
7,99407b12-f463-4b99-a420-2962667d0fef,9188c8b2-05a5-4379-9c5c-3673db14465e,Id
11,cd655441-bac8-4d0b-9c8c-56233814fbb2,e2493d15-0654-4abd-8e80-4230d8cb6bd5,Pâture
15,9695ae1d-09e7-4493-8a10-0c536d4b53d6,a8029037-9171-4573-a275-d11f5c898baf,Id
19,c69e476c-065d-4833-8730-11fa475e280e,9786aa20-6e74-493c-85e0-803ad682a5db,Bois
...,...,...,...
2132,5f4cdf48-9c20-4d7f-ac13-bd3fbe08503d,eb494ccc-a4ae-42da-8cc4-a1c2b9178a8b,id
2136,4a186c8f-51db-482b-a129-7f9a1d87364e,ac751d31-bebb-496e-99ea-a770b2699af7,id
2140,fce30d28-1fd2-4d57-9bdd-f4b990a936fe,f42cf3b1-abf7-443a-ad6f-bf295bb144b0,id
2144,33680098-0604-4637-a891-675c9cee8878,68fed831-df5b-47bc-9fbf-5211b9fd5c09,Terre


In [ ]:
lieu_p = cleantab[cleantab['entity_type'] == 'lieu-dit']
lieup_select = lieu_p[['lineid','entity_id','entity_text']]
lieup_select = lieup_select.rename(columns={"entity_id": "lieu-dit_srcid", "entity_text": "lieu-dit"})
display(lieup_select)

,lineid,lieu-dit_srcid,lieu-dit
0,e92ddd2b-583f-4234-8a52-9f8745cda6c2,5230b28e-0fd9-403a-8cd9-ba8d3428a91f,Bois au pont→d'Encre
4,99407b12-f463-4b99-a420-2962667d0fef,09c877bf-10c0-4913-b316-0b8977ed3ce3,§
8,cd655441-bac8-4d0b-9c8c-56233814fbb2,0367b5d4-cdeb-4650-86c9-2b26a2a24109,Bois de la→Grippe
12,9695ae1d-09e7-4493-8a10-0c536d4b53d6,28fb3e63-59c8-4cf1-bf6a-e893bdb6a2fb,Les uzelles de→Marolles
16,c69e476c-065d-4833-8730-11fa475e280e,8788923a-7403-40c2-9273-02408bc72a78,Bois de la→grippe
...,...,...,...
2129,5f4cdf48-9c20-4d7f-ac13-bd3fbe08503d,a213b8fa-f32d-4021-a13d-f5fda39eb71c,§
2133,4a186c8f-51db-482b-a129-7f9a1d87364e,bbfc573e-b16a-4bc3-8d27-06ba090263c6,§
2137,fce30d28-1fd2-4d57-9bdd-f4b990a936fe,c1535d2d-493b-4c09-ba1a-368653bf2539,§
2141,33680098-0604-4637-a891-675c9cee8878,fca7a0de-3de0-4c5d-8652-c910e59aec4e,§


In [ ]:
propnum_p = cleantab[cleantab['entity_type'] == 'numero_proprietaire']
propnump_select = propnum_p[['lineid','entity_id','entity_text']]
propnump_select = propnump_select.rename(columns={"entity_id": "numero_proprietaire_srcid", "entity_text": "numero_proprietaire"})
display(propnump_select)

,lineid,numero_proprietaire_srcid,numero_proprietaire
1096,06a59e14-2e8b-4ba6-af46-a17a1daa9ce8,b2d4da73-e28e-4d10-ba1f-f0f7c04e16a4,70
1101,dc8bea78-d114-4dc2-bc7e-275fc69a213a,93f9da23-a62d-40e2-95e8-405642aed4ca,34


## Création du tableau final

In [ ]:
finaltab = cleantab[['section_id','section_name','page_cote','page_cote_num','lineid','linename','coords','transcriptid','text','text_confidence','entities_count','image_id',
               'image_name','image_url','image_width','image_height']]
finaltab = finaltab.drop_duplicates(subset=['lineid'])
finaltab.reset_index()
finaltab

,section_id,section_name,page_cote,page_cote_num,lineid,linename,coords,transcriptid,text,text_confidence,entities_count,image_id,image_name,image_url,image_width,image_height
0,A,dite des Bruyères,FRAD094_3P_000387_01_0003,3,e92ddd2b-583f-4234-8a52-9f8745cda6c2,1,"[[223, 792], [223, 1015], [2520, 1015], [2520,...",55f95cd0-786d-44b2-8ddb-c59846c72b2e,Bois au pont→d'Encre 1 Le Prince de Neuchâtel→...,1.0,4,c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,4013
4,A,dite des Bruyères,FRAD094_3P_000387_01_0003,3,99407b12-f463-4b99-a420-2962667d0fef,2,"[[219, 973], [219, 1129], [2525, 1129], [2525,...",a5e9469a-1cc3-4d44-bec9-062573e034f1,§ 2 Idem Id,1.0,4,c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,4013
8,A,dite des Bruyères,FRAD094_3P_000387_01_0003,3,cd655441-bac8-4d0b-9c8c-56233814fbb2,3,"[[219, 1110], [219, 1285], [2518, 1285], [2518...",0cba8885-adb6-43c3-896c-c3307413a37c,Bois de la→Grippe 3 Domaine de la→Couronne Pâture,1.0,4,c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,4013
12,A,dite des Bruyères,FRAD094_3P_000387_01_0003,3,9695ae1d-09e7-4493-8a10-0c536d4b53d6,4,"[[215, 1272], [215, 1393], [2518, 1393], [2518...",df0d2251-755f-48d2-a15d-ecc239d3d684,Les uzelles de→Marolles 4 Idem Id,1.0,4,c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,4013
16,A,dite des Bruyères,FRAD094_3P_000387_01_0003,3,c69e476c-065d-4833-8730-11fa475e280e,5,"[[222, 1387], [222, 1562], [2515, 1562], [2515...",9b69ba2d-0d03-441d-8b93-fabe8e92ddbf,Bois de la→grippe 5 Idem Bois,1.0,4,c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,4013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2129,D,dite du pont de brie,FRAD094_3P_000387_01_0100,100,5f4cdf48-9c20-4d7f-ac13-bd3fbe08503d,5,"[[326, 1369], [326, 1570], [2636, 1570], [2636...",00efbad7-ab5a-40ec-8dae-af49ab68f33c,§ 225 Coudray auguste→andré id,1.0,4,302729ca-34b8-429d-8d32-cf8da9995c98,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2787,4082
2133,D,dite du pont de brie,FRAD094_3P_000387_01_0100,100,4a186c8f-51db-482b-a129-7f9a1d87364e,6,"[[329, 1527], [329, 1683], [2639, 1683], [2639...",adbd7fbb-53b5-4a9e-b088-1dbe21d82ea4,§ 226 huguet Louis id,1.0,4,302729ca-34b8-429d-8d32-cf8da9995c98,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2787,4082
2137,D,dite du pont de brie,FRAD094_3P_000387_01_0100,100,fce30d28-1fd2-4d57-9bdd-f4b990a936fe,7,"[[326, 1651], [326, 1796], [2633, 1796], [2633...",5415ac66-4c66-42cc-8f84-8d67e2f193b0,§ 227 Berthier id,1.0,4,302729ca-34b8-429d-8d32-cf8da9995c98,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2787,4082
2141,D,dite du pont de brie,FRAD094_3P_000387_01_0100,100,33680098-0604-4637-a891-675c9cee8878,8,"[[320, 1774], [320, 1913], [2633, 1913], [2633...",8f972b45-ff29-430b-92da-8dfc853dab18,§ 228 Tartet Veuve Terre,1.0,4,302729ca-34b8-429d-8d32-cf8da9995c98,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2787,4082


In [ ]:
finaltab0 = pd.merge(finaltab, nump_select, on=["lineid"],how="outer")
finaltab1 = pd.merge(finaltab0, lieup_select, on=["lineid"],how="outer")
finaltab2 = pd.merge(finaltab1, propnump_select, on=["lineid"],how="outer")
finaltab3 = pd.merge(finaltab2, identitep_select, on=["lineid"],how="outer")
finaltab4 = pd.merge(finaltab3, naturep_select, on=["lineid"],how="outer")
display(finaltab4)

,section_id,section_name,page_cote,page_cote_num,lineid,linename,coords,transcriptid,text,text_confidence,...,numero_parcelle_srcid,numero_parcelle,lieu-dit_srcid,lieu-dit,numero_proprietaire_srcid,numero_proprietaire,proprietaire_srcid,proprietaire,nature_srcid,nature
0,A,dite des Bruyères,FRAD094_3P_000387_01_0003,3,e92ddd2b-583f-4234-8a52-9f8745cda6c2,1,"[[223, 792], [223, 1015], [2520, 1015], [2520,...",55f95cd0-786d-44b2-8ddb-c59846c72b2e,Bois au pont→d'Encre 1 Le Prince de Neuchâtel→...,1.0,...,1268c53b-962f-450e-8713-6cf7d4aa2f88,1,5230b28e-0fd9-403a-8cd9-ba8d3428a91f,Bois au pont→d'Encre,NaN,NaN,c19adf3a-7888-4363-804d-c3161806f0df,"Le Prince de Neuchâtel→×Vauguyon→Bourgeois, ma...",e1b7d3aa-89a1-4218-91ff-b75e678e7de5,Bois
1,A,dite des Bruyères,FRAD094_3P_000387_01_0003,3,99407b12-f463-4b99-a420-2962667d0fef,2,"[[219, 973], [219, 1129], [2525, 1129], [2525,...",a5e9469a-1cc3-4d44-bec9-062573e034f1,§ 2 Idem Id,1.0,...,3f59a7ee-5556-4938-809f-ec6fde6c1c62,2,09c877bf-10c0-4913-b316-0b8977ed3ce3,§,NaN,NaN,27acbd42-bfce-46d9-afec-856de6a9cbcd,Idem,9188c8b2-05a5-4379-9c5c-3673db14465e,Id
2,A,dite des Bruyères,FRAD094_3P_000387_01_0003,3,cd655441-bac8-4d0b-9c8c-56233814fbb2,3,"[[219, 1110], [219, 1285], [2518, 1285], [2518...",0cba8885-adb6-43c3-896c-c3307413a37c,Bois de la→Grippe 3 Domaine de la→Couronne Pâture,1.0,...,1fcf647c-61cd-4542-b38b-541ede5be2af,3,0367b5d4-cdeb-4650-86c9-2b26a2a24109,Bois de la→Grippe,NaN,NaN,7577a18e-e2d9-415b-b28f-eed31a3dbe9f,Domaine de la→Couronne,e2493d15-0654-4abd-8e80-4230d8cb6bd5,Pâture
3,A,dite des Bruyères,FRAD094_3P_000387_01_0003,3,9695ae1d-09e7-4493-8a10-0c536d4b53d6,4,"[[215, 1272], [215, 1393], [2518, 1393], [2518...",df0d2251-755f-48d2-a15d-ecc239d3d684,Les uzelles de→Marolles 4 Idem Id,1.0,...,845c9397-711e-4973-afdb-7010014da54a,4,28fb3e63-59c8-4cf1-bf6a-e893bdb6a2fb,Les uzelles de→Marolles,NaN,NaN,d046417b-0a6d-4ada-8653-88d211425f53,Idem,a8029037-9171-4573-a275-d11f5c898baf,Id
4,A,dite des Bruyères,FRAD094_3P_000387_01_0003,3,c69e476c-065d-4833-8730-11fa475e280e,5,"[[222, 1387], [222, 1562], [2515, 1562], [2515...",9b69ba2d-0d03-441d-8b93-fabe8e92ddbf,Bois de la→grippe 5 Idem Bois,1.0,...,1f268a1d-fc4a-483c-82bd-6a30590ded95,5,8788923a-7403-40c2-9273-02408bc72a78,Bois de la→grippe,NaN,NaN,0d468201-166f-49ef-ae12-f970b9c7de94,Idem,9786aa20-6e74-493c-85e0-803ad682a5db,Bois
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,D,dite du pont de brie,FRAD094_3P_000387_01_0100,100,5f4cdf48-9c20-4d7f-ac13-bd3fbe08503d,5,"[[326, 1369], [326, 1570], [2636, 1570], [2636...",00efbad7-ab5a-40ec-8dae-af49ab68f33c,§ 225 Coudray auguste→andré id,1.0,...,0a2dc928-caba-4fb7-a9e3-6f2daa8c7e0e,225,a213b8fa-f32d-4021-a13d-f5fda39eb71c,§,NaN,NaN,955b6f67-d7bb-4819-807b-c0d357c1830d,Coudray auguste→andré,eb494ccc-a4ae-42da-8cc4-a1c2b9178a8b,id
533,D,dite du pont de brie,FRAD094_3P_000387_01_0100,100,4a186c8f-51db-482b-a129-7f9a1d87364e,6,"[[329, 1527], [329, 1683], [2639, 1683], [2639...",adbd7fbb-53b5-4a9e-b088-1dbe21d82ea4,§ 226 huguet Louis id,1.0,...,aac242a1-9f27-4562-af98-4c396d2fc95d,226,bbfc573e-b16a-4bc3-8d27-06ba090263c6,§,NaN,NaN,18f05b43-fd40-49d8-ad7c-95f947ef45d0,huguet Louis,ac751d31-bebb-496e-99ea-a770b2699af7,id
534,D,dite du pont de brie,FRAD094_3P_000387_01_0100,100,fce30d28-1fd2-4d57-9bdd-f4b990a936fe,7,"[[326, 1651], [326, 1796], [2633, 1796], [2633...",5415ac66-4c66-42cc-8f84-8d67e2f193b0,§ 227 Berthier id,1.0,...,446ba364-32a7-443f-a1b7-ecedf0898a8d,227,c1535d2d-493b-4c09-ba1a-368653bf2539,§,NaN,NaN,c8a78be8-59e2-4461-ab5a-35b5ba3147f1,Berthier,f42cf3b1-abf7-443a-ad6f-bf295bb144b0,id
535,D,dite du pont de brie,FRAD094_3P_000387_01_0100,100,33680098-0604-4637-a891-675c9cee8878,8,"[[320, 1774], [320, 1913], [2633, 1913], [2633...",8f972b45-ff29-430b-92da-8dfc853dab18,§ 228 Tartet Veuve Terre,1.0,...,3bab9126-43a5-4538-84de-3717d7787690,228,fca7a0de-3de0-4c5d-8652-c910e59aec4e,§,NaN,NaN,38cd34fa-b3fc-4a42-82f5-e1157b3afc4b,Tartet Veuve,68fed

In [ ]:
finaltab4.columns

Index(['section_id', 'section_name', 'page_cote', 'page_cote_num', 'lineid',
       'linename', 'coords', 'transcriptid', 'text', 'text_confidence',
       'entities_count', 'image_id', 'image_name', 'image_url', 'image_width',
       'image_height', 'numero_parcelle_srcid', 'numero_parcelle',
       'lieu-dit_srcid', 'lieu-dit', 'numero_proprietaire_srcid',
       'numero_proprietaire', 'proprietaire_srcid', 'proprietaire',
       'nature_srcid', 'nature'],
      dtype='object')

In [ ]:
finalversions = finaltab4[['section_id', 'section_name', 'lieu-dit','numero_parcelle','nature','proprietaire','numero_proprietaire','text',
       'page_cote','page_cote_num', 'lineid','linename','entities_count','coords', 'transcriptid', 'text_confidence',
       'image_height', 'image_id', 'image_name', 'image_url', 'image_width',
       'numero_parcelle_srcid', 'lieu-dit_srcid',  'numero_proprietaire_srcid','proprietaire_srcid', 'nature_srcid']]

In [ ]:
coords_ranking = finalversions["coords"]

ccrr =[]
for c in coords_ranking:
  rank = c[0][1]
  ccrr.append(rank)

finalversions["coords_ranking"] = ccrr
finalversions = finalversions.sort_values(by=['section_id','page_cote_num','coords_ranking'])

<ipython-input-28-fe4b9442f414>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalversions["coords_ranking"] = ccrr


In [ ]:
display(finalversions)

,section_id,section_name,lieu-dit,numero_parcelle,nature,proprietaire,numero_proprietaire,text,page_cote,page_cote_num,...,image_id,image_name,image_url,image_width,numero_parcelle_srcid,lieu-dit_srcid,numero_proprietaire_srcid,proprietaire_srcid,nature_srcid,coords_ranking
0,A,dite des Bruyères,Bois au pont→d'Encre,1,Bois,"Le Prince de Neuchâtel→×Vauguyon→Bourgeois, ma...",NaN,Bois au pont→d'Encre 1 Le Prince de Neuchâtel→...,FRAD094_3P_000387_01_0003,3,...,c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,1268c53b-962f-450e-8713-6cf7d4aa2f88,5230b28e-0fd9-403a-8cd9-ba8d3428a91f,NaN,c19adf3a-7888-4363-804d-c3161806f0df,e1b7d3aa-89a1-4218-91ff-b75e678e7de5,792
1,A,dite des Bruyères,§,2,Id,Idem,NaN,§ 2 Idem Id,FRAD094_3P_000387_01_0003,3,...,c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,3f59a7ee-5556-4938-809f-ec6fde6c1c62,09c877bf-10c0-4913-b316-0b8977ed3ce3,NaN,27acbd42-bfce-46d9-afec-856de6a9cbcd,9188c8b2-05a5-4379-9c5c-3673db14465e,973
2,A,dite des Bruyères,Bois de la→Grippe,3,Pâture,Domaine de la→Couronne,NaN,Bois de la→Grippe 3 Domaine de la→Couronne Pâture,FRAD094_3P_000387_01_0003,3,...,c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,1fcf647c-61cd-4542-b38b-541ede5be2af,0367b5d4-cdeb-4650-86c9-2b26a2a24109,NaN,7577a18e-e2d9-415b-b28f-eed31a3dbe9f,e2493d15-0654-4abd-8e80-4230d8cb6bd5,1110
3,A,dite des Bruyères,Les uzelles de→Marolles,4,Id,Idem,NaN,Les uzelles de→Marolles 4 Idem Id,FRAD094_3P_000387_01_0003,3,...,c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,845c9397-711e-4973-afdb-7010014da54a,28fb3e63-59c8-4cf1-bf6a-e893bdb6a2fb,NaN,d046417b-0a6d-4ada-8653-88d211425f53,a8029037-9171-4573-a275-d11f5c898baf,1272
4,A,dite des Bruyères,Bois de la→grippe,5,Bois,Idem,NaN,Bois de la→grippe 5 Idem Bois,FRAD094_3P_000387_01_0003,3,...,c4d1864d-5e5f-4258-bc06-8addd9464d72,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2665,1f268a1d-fc4a-483c-82bd-6a30590ded95,8788923a-7403-40c2-9273-02408bc72a78,NaN,0d468201-166f-49ef-ae12-f970b9c7de94,9786aa20-6e74-493c-85e0-803ad682a5db,1387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,D,dite du pont de brie,§,225,id,Coudray auguste→andré,NaN,§ 225 Coudray auguste→andré id,FRAD094_3P_000387_01_0100,100,...,302729ca-34b8-429d-8d32-cf8da9995c98,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2787,0a2dc928-caba-4fb7-a9e3-6f2daa8c7e0e,a213b8fa-f32d-4021-a13d-f5fda39eb71c,NaN,955b6f67-d7bb-4819-807b-c0d357c1830d,eb494ccc-a4ae-42da-8cc4-a1c2b9178a8b,1369
533,D,dite du pont de brie,§,226,id,huguet Louis,NaN,§ 226 huguet Louis id,FRAD094_3P_000387_01_0100,100,...,302729ca-34b8-429d-8d32-cf8da9995c98,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2787,aac242a1-9f27-4562-af98-4c396d2fc95d,bbfc573e-b16a-4bc3-8d27-06ba090263c6,NaN,18f05b43-fd40-49d8-ad7c-95f947ef45d0,ac751d31-bebb-496e-99ea-a770b2699af7,1527
534,D,dite du pont de brie,§,227,id,Berthier,NaN,§ 227 Berthier id,FRAD094_3P_000387_01_0100,100,...,302729ca-34b8-429d-8d32-cf8da9995c98,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/iiif/2/CADA...,2787,446ba364-32a7-443f-a1b7-ecedf0898a8d,c1535d2d-493b-4c09-ba1a-368653bf2539,NaN,c8a78be8-59e2-4461-ab5a-35b5ba3147f1,f42cf3b1-abf7-443a-ad6f-bf295bb144b0,1651
535,D,dite du pont de brie,§,228,Terre,Tartet Veuve,NaN,§ 228 Tartet Veuve Terre,FRAD094_3P_000387_01_0100,100,...,302729ca-34b8-429d-8d32-cf8da9995c98,CADASTRE%2FETATS_DE_SECTION%2F%2FMAROLLES%2FFR...,https://iiif.geohistoricaldata.org/i

In [ ]:
finalversions.to_csv(path_or_buf='/content/drive/MyDrive/THESE/ONTOLOGIE/marolles_en_brie_data.csv',index=False,encoding='utf8',sep='\t')